In [ ]:
from pyspark.sql import SparkSession
import glob
import numpy as np
import pandas as pd
import os
import concurrent.futures
from tqdm import tqdm

In [ ]:
#Set Environment accordingly
os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home/"
os.environ['SPARK_HOME'] = "/Users/simran/Downloads/spark-3.5.5-bin-hadoop3/"

In [ ]:
!wget "https://huggingface.co/datasets/Zihan1004/FNSPID/resolve/main/Stock_price/full_history.zip"
!unzip "./full_history.zip" -d "./"

In [ ]:
directory_path = "./full_history"
file_pattern = "*.csv"

In [ ]:
#Preprocess Data before reading to Spark
desired_order = ['date','volume','open','high','low','close','adj close']
new_order = ['date','volume','open','high','low','close','adj close','StockName']
def amendCSV(filename):
  df = pd.read_csv(filename, usecols=desired_order)
  stockName = filename
  df['StockName'] = stockName.split("/")[-1].split(".csv")[0]  # Add filename column
  df.to_csv(filename,index=False,columns=new_order)
  return 1

all_files = glob.glob(os.path.join(directory_path, file_pattern), recursive=True)
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
  result = list(tqdm(executor.map(amendCSV, all_files), total = len(all_files)))
print('All done.')